In [ ]:
#hide

# RaDICaL Dataset SDK

> Summary description here.

This file will become your README and also the index of your documentation.

## Install

`pip install radical-dataset`

## How to use

Fill me in please! Don't forget code examples: